# Lesson 3 Demo 4: Using the WHERE Clause
<img src="../../../images/cassandra_logo.png" width="250" height="250">

In [48]:
from src.utils.cassandra.generic_commands import CassandraCommands

#### Create a connection to the database

In [49]:
cassandra = CassandraCommands()
cassandra.connect('127.0.0.1')

CassandraCommands class initiated
Connection setup with cassandra at 127.0.0.1


#### Create a keyspace to do our work in 

In [50]:
cassandra.create_keyspace('data_engeneering_nano_degree')
cassandra.connect_keyspace('data_engeneering_nano_degree')

Keyspace named data_engeneering_nano_degree created
Connected to keyspace: data_engeneering_nano_degree


#### Let's imagine we would like to start creating a new Music Library of albums. 
We want to ask 4 question of our data
##### 1. Give me every album in my music library that was released in a 1965 year
##### 2. Give me the album that is in my music library that was released in 1965 by "The Beatles"
##### 3. Give me all the albums released in a given year that was made in London 
##### 4. Give me the city that the album "Rubber Soul" was recorded

### Here is our Collection of Data
<img src="../../../images/primary_key.png" width="650" height="350">

#### How should we model this data? What should be our Primary Key and Partition Key? Since our data is looking for the YEAR let's start with that. From there we will add clustering columns on Artist Name and Album Name.

In [57]:
cassandra.drop_table(table_name='music_library_partition_1')
cassandra.create_table(table_name='music_library_partition_1', schema='(year int, city varchar, artist_name varchar, album_name varchar, PRIMARY KEY (year, artist_name))')

Error from server: code=2200 [Invalid query] message="Table 'data_engeneering_nano_degree.music_library_partition_1' doesn't exist"
Table named music_library_partition_1 created


In [58]:
cassandra.drop_table(table_name='music_library_partition_2')
cassandra.create_table(table_name='music_library_partition_2', schema='(year int, city varchar, artist_name varchar, album_name varchar, PRIMARY KEY (year, city, artist_name))')

Dropped table music_library_partition_2
Table named music_library_partition_2 created


In [59]:
cassandra.drop_table(table_name='music_library_partition_3')
cassandra.create_table(table_name='music_library_partition_3', schema='(year int, city varchar, artist_name varchar, album_name varchar, PRIMARY KEY (album_name))')

Dropped table music_library_partition_3
Table named music_library_partition_3 created


#### Let's insert our data into of table

In [60]:
for i in range(1,4):
    cassandra.insert_rows(table_name=f'music_library_partition_{i}', 
                          columns='(year, city, artist_name, album_name)',
                          rows=[
                               (1965, 'Oxford', 'The Beatles', 'Rubber Soul'),
                               (1970, 'Liverpool', 'The Beatles', 'Let it Be'),
                               (1966, 'Los Angeles', 'The Monkees', 'The Monkees'), 
                               (1970, 'San Diego', 'The Carpenters', 'Close To You'),
                               (1965, 'London', 'The Who', 'My Generation'),
                               ]
                         )              

#### Let's Validate our Data Model with our 4 queries.

##### 1. Give me every album in my music library that was released in a 1965 year

In [61]:
q = "select * from music_library_partition_1 WHERE year=1965"
cassandra.custom_query(query=q)

[Row(year=1965, artist_name='The Beatles', album_name='Rubber Soul', city='Oxford'),
 Row(year=1965, artist_name='The Who', album_name='My Generation', city='London')]

##### 2. Give me the album that is in my music library that was released in 1965 by "The Beatles"

In [62]:
q = "select * from music_library_partition_1 WHERE year=1965 AND artist_name='The Beatles'"
cassandra.custom_query(query=q)

[Row(year=1965, artist_name='The Beatles', album_name='Rubber Soul', city='Oxford')]

##### 3. Give me all the albums released in a given year that was made in London 

In [43]:
q = "select artist_name, album_name from music_library_partition_2 WHERE year=1965 AND city='London'"
cassandra.custom_query(query=q)

[Row(artist_name='The Who', album_name='My Generation')]

##### 4. Give me the city that the album "Rubber Soul" was recorded

In [45]:
q = "select city from music_library_partition_3 WHERE album_name = 'Rubber Soul'"
cassandra.custom_query(query=q)

[Row(city='Oxford')]

#### Close the session and cluster connection

In [46]:
cassandra.close_connection()

Closed session
Closed cluster connection
